In [ ]:
# === Beschreibung ===
# Dieses Programm verarbeitet Mietdaten aus Deutschland, den USA und Indien für das Jahr 2024.
# Ziel ist es, die Mietpreisentwicklung der drei Länder anhand offizieller Indizes zu vergleichen.
# Die Daten stammen aus öffentlich zugänglichen Quellen (ECB, FRED, RBI) und werden vereinheitlicht, normalisiert und visualisiert.

# === Verarbeitung von Mietdaten für Deutschland, die USA und Indien ===

# 1. Benötigte Bibliotheken installieren:
!pip install pandas numpy plotly openpyxl

# 2. Bibliotheken importieren:
import pandas as pd                  # Für Datenverarbeitung
import numpy as np                   # Für numerische Operationen wie Interpolation
import plotly.express as px          # Für interaktive Diagramme
from google.colab import files       # Für Datei-Upload und -Download in Google Colab

# 3. Lokale CSV-Dateien hochladen:
print("Bitte laden Sie die folgenden Dateien hoch:\n- Composite HPI for 50 Cities.csv\n- ECB Data Portal_20250612142210.csv\n- CUUR0000SEHA .csv")  # Hinweis für Dateinamen
uploaded = files.upload()            # Benutzer wählt die 3 Dateien manuell aus

# 4. CSV-Dateien einlesen:
india_df = pd.read_csv("Composite HPI for 50 Cities.csv")                        # Indien (quartalsweise HPI)
germany_df = pd.read_csv("ECB Data Portal_20250612142210.csv")                  # Deutschland (monatlicher Mietindex)
usa_df = pd.read_csv("CUUR0000SEHA .csv")                                       # USA (monatlicher Mietindex)

# 5. Deutschland: Zeitformat harmonisieren (Monatsende zu Monatsanfang):
germany_df = germany_df.rename(columns={
    'HICP - Actual rentals for housing (ICP.M.DE.N.041000.4.INX)': 'Germany_Rent_Index',
    'DATE': 'Datum'   # Spaltenname geändert zu "Datum"
})
germany_df['Datum'] = pd.to_datetime(germany_df['Datum'], errors='coerce')        # Umwandlung in Datumsformat
germany_df['Datum'] = germany_df['Datum'].dt.to_period('M').dt.to_timestamp()     # Monatsanfang extrahieren
germany_df['Datum'] = germany_df['Datum'].dt.strftime("%d/%m/%Y")                 # Datumsformat anpassen: TT/MM/JJJJ
germany_2024 = germany_df[germany_df['Datum'].str.endswith("/2024")].copy()       # Nur Daten für 2024

# 6. USA: Daten vorbereiten:
usa_df['observation_date'] = pd.to_datetime(usa_df['observation_date'], errors='coerce')    # Datumsformat
usa_df['Datum'] = usa_df['observation_date'].dt.strftime("%d/%m/%Y")                        # Neues Datumsformat hinzufügen
usa_2024 = usa_df[usa_df['Datum'].str.endswith("/2024")].copy()                             # Nur 2024
usa_2024 = usa_2024.rename(columns={
    'CUUR0000SEHA': 'USA_Rent_Index'
})

# 7. Indien: Quartalsdaten interpolieren in Monatsdaten:
india_2024 = india_df[india_df['Quarter'].str.contains("2024", na=False)].copy()
india_2024['Quarter_Label'] = india_2024['Quarter'].str[:3].map({'Mar': 'Q1', 'Jun': 'Q2', 'Sep': 'Q3', 'Dec': 'Q4'})
india_2024['HPI'] = india_2024['HPI@Market Prices for Under Construction Properties'].astype(str).str.strip()
india_2024['HPI'] = india_2024['HPI'].replace('', np.nan).astype(float)
india_2024 = india_2024.dropna(subset=['Quarter_Label', 'HPI'])                  # Nur vollständige Daten

# Schätzung: Falls Q4 fehlt, wird Q4 aus Q2 & Q3 extrapoliert
available_qs = india_2024['Quarter_Label'].tolist()
if 'Q4' not in available_qs and 'Q2' in available_qs and 'Q3' in available_qs:
    q2 = india_2024.set_index('Quarter_Label').loc['Q2', 'HPI']
    q3 = india_2024.set_index('Quarter_Label').loc['Q3', 'HPI']
    q4 = q3 + (q3 - q2)            # Lineare Extrapolation
    india_2024 = pd.concat([india_2024, pd.DataFrame([{'Quarter_Label': 'Q4', 'HPI': q4}])])

# Interpolation der Monatswerte für Indien:
india_2024 = india_2024.set_index('Quarter_Label').sort_index()
quarterly_values = india_2024['HPI'].to_dict()
months = pd.date_range(start="2024-01-01", end="2024-12-01", freq='MS')   # Monatsanfangsliste
interpolated = []

# Interpolation zwischen den Quartalen (12 Monatswerte):
for i in range(3):
    interpolated.extend(np.linspace(quarterly_values[f'Q{i+1}'], quarterly_values[f'Q{i+2}'], 4)[:-1])
interpolated.extend(np.linspace(quarterly_values['Q3'], quarterly_values['Q4'], 4)[1:])

# Monatstexte im gewünschten Format erzeugen:
months_str = months.strftime("%d/%m/%Y")   # Datumsformat TT/MM/JJJJ für Indien

# 8. Zusammenführen aller Länder in einem DataFrame:
df_germany = germany_2024[['Datum', 'Germany_Rent_Index']].rename(columns={'Germany_Rent_Index': 'Deutschland'})
df_usa = usa_2024[['Datum', 'USA_Rent_Index']].rename(columns={'USA_Rent_Index': 'USA'})
df_india = pd.DataFrame({'Datum': months_str, 'Indien (Interpoliert)': interpolated})  # Spaltenname von Anfang an als "Datum"

# Merge aller drei Länder nach Datum:
merged = df_germany.merge(df_usa, on='Datum').merge(df_india, on='Datum')

# 9. Normalisierung: Alle Werte starten bei 100 im Januar:
for col in ['Deutschland', 'USA', 'Indien (Interpoliert)']:
    merged[col] = (merged[col] / merged[col].iloc[0]) * 100
    merged[col] = merged[col].round(1)     # Auf eine Nachkommastelle runden

# 10. Excel-Datei exportieren:
merged.to_excel("Processed_Rent_Data_2024.xlsx", index=False)
files.download("Processed_Rent_Data_2024.xlsx")


# === VISUALISIERUNG DER EXPORTIERTEN DATEN FÜR DIE MIETPREISENTWICKLUNG ===

# Daten in Langformat bringen für Plotly:
df_long = merged.melt(id_vars='Datum', var_name='Country', value_name='Rent_Index')  # Für farbige Linien

# Interaktives Liniendiagramm erstellen:
fig = px.line(
    df_long,
    x='Datum',
    y='Rent_Index',
    color='Country',
    title='Mietpreisentwicklung 2024 (Januar = 100)',
    labels={'Datum': 'Monat', 'Rent_Index': 'Indexwert', 'Country': 'Land'},
    markers=True
)

# Hovereffekte und Layout anpassen:
fig.update_traces(mode="lines+markers", hovertemplate='%{x}: %{y:.1f}')
fig.update_layout(
    width=1000,
    height=400,
    hovermode="x unified",
    xaxis=dict(title="Monat", tickangle=0),
    yaxis=dict(title="Indexwert (relativ zu Januar)", tickformat=".1f"),
    template="simple_white"
)

# Diagramm anzeigen:
fig.show()
